# B. Sim Dashboard

### Imports

In [1]:
%run "C:\Users\james\Documents\MLB\Code\U1. Imports.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U2. Utilities.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U3. Classes.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U4. Datasets.ipynb"

In [2]:
# Schedule
pause_code(start_time='2024-08-03T18:00:00', timezone='EST')

Time until 06:00PM. -72 hours, 35 minutes, and 58 seconds.


### Games

Select start and end date (should be todaysdate for both if running day-of)

In [3]:
start_date = "20240320"
end_date = yesterdaysdate

start_date = todaysdate
end_date = todaysdate

Read in games

In [4]:
game_df = read_and_save_games(team_map, generate=True)
game_df = game_df[(game_df['date'] >= start_date) & (game_df['date'] <= end_date)].reset_index(drop=True)

### Settings

In [5]:
# Run historic matchups
historic = len(game_df['date'].unique()) > 1
# Run matchups necessary for slate
slate_only = not historic 

In [6]:
%%time
if historic == False:
    %run "A. Night Dashboard.ipynb"

Running Night Dashboard
CPU times: total: 0 ns
Wall time: 49.7 s
CPU times: total: 31.2 ms
Wall time: 47.4 s
CPU times: total: 31.2 ms
Wall time: 2.99 s
20240806
CHW@OAK uses Swish Analytics weather.
PHI@LAD uses Swish Analytics weather.
KeyError
Scraping successful!
Contest Guide 164596607 is empty.
Contest Guide 164596609 is empty.
Contest Guide 164596603 is empty.
Contest Guide 164595486 is empty.
Contest Guide 164594576 is empty.
Contest Guide 164596608 is empty.
CPU times: total: 9.3 s
Wall time: 2min 14s


### Contest

In [7]:
# Read in subset of contests for which we'll create guides
subset_df = pd.read_csv(os.path.join(baseball_path, 'A01. DraftKings', '7. Subsets', f'Subset {todaysdate}.csv'))
subset_df['contestDate'] = pd.to_datetime(subset_df['contestDate'])
subset_df = subset_df[(subset_df['Name'].str.contains('oonshot')) & ~(subset_df['Name'].str.contains('Turbo')) & ~(subset_df['Name'].str.contains('@'))].sort_values('contestDate', ascending=True).reset_index(drop=True)
subset_df

,Name,Cash Prize,Entry Fee,contestKey,draftGroupId,contestDate,contestTime,date,date_dash
0,MLB $30K Moonshot [20 Entry Max],30000.0,3.0,164597148,111005,2024-08-06 19:05:00,Tue 7:05PM,20240806,2024-08-06


##### Select Contest

In [8]:
# Set contestKey
contestKey = subset_df['contestKey'][0]

In [9]:
# Read in Contest Guide
guide = pd.read_csv(os.path.join(baseball_path, "A09. Contest Guides", f"Contest Guide {contestKey}.csv"))

# Identify draftGroupId, RotoWire slate
draftGroupId, roto_slate = guide['draftGroupId'][0], guide['roto_slate'][0]

# Print out name
print(f"Date: {guide['date'][0]}. Slate: {guide['slate'][0]}. {guide['name'][0]}. {roto_slate}.")

Date: 20240806. Slate: All. MLB $30K Moonshot [20 Entry Max]. 17688.


In [10]:
if slate_only == True:
    slate_df = game_df[game_df['game_id'].isin(guide['game_id'].unique())].reset_index(drop=True)

###### 

### Weather

In [11]:
try:
    rotogrinders_df = pd.read_csv(os.path.join(baseball_path, "A06. Weather", "2. RotoGrinders", f"RotoGrinders {todaysdate}.csv"))
    red_list = list(rotogrinders_df.query('Tag == "red" or Tag2 == "red"')['Away']) + list(rotogrinders_df.query('Tag == "red" or Tag2 == "red"')['Home'])
    orange_list = list(rotogrinders_df.query('Tag == "orange" or Tag2 == "orange"')['Away']) + list(rotogrinders_df.query('Tag == "orange" or Tag2 == "orange"')['Home'])
    display_weather = rotogrinders_df[['Tag', 'Tag2', 'Away', 'Home', 'date', 'Description']]
except:
    red_list, orange_list, display_weather = [], [], None
    
display_weather

,Tag,Tag2,Away,Home,date,Description
0,orange,yellow,ARI,CLE,20240806,"A nasty line of storms has cleared CLE, and that is the last truly ugly stuff they'll see tonight. There is however still a good deal of scattered light-to-moderate rain still upstream, which will gradually fade out through the night. They could try to play through it and risk a delay, or late start and try to play dry... it's possible they just avoid all of the drama and PPD it. Bottom line - don't know yet."
1,orange,NaN,SD,PIT,20240806,"The worst of the weather should be around first pitch, and it does look quite ugly. They could late start and play it dry'ish if they wait a couple hours."
2,yellow,NaN,SF,WSH,20240806,A weakening batch of rain tries to move in to DC in the mid-late innings. Chance for a delay late.
3,red,NaN,LAA,NYY,20240806,PPD in NY.
4,green,NaN,MIN,CHC,20240806,"Some light drizzle possible. Winds ripping in from left-center at 20mph, big help to SP's."
5,green,yellow,NYM,COL,20240806,"I won't fully rule out a late start, but I don't expect one. Winds out to center at 10mph."


### B01. Matchups

In [12]:
%run "C:\Users\james\Documents\MLB\Code\B01. Matchups.ipynb"

##### Load
This loads in datasets needed to create matchup files

In [13]:
%%time
complete_dataset = pd.read_csv(os.path.join(baseball_path, "Complete Dataset.csv"))
steamer_hitters_df = pd.read_csv(os.path.join(baseball_path, "Steamer Hitters.csv"))
steamer_pitchers_df = pd.read_csv(os.path.join(baseball_path, "Steamer Pitchers.csv"))

CPU times: total: 18.7 s
Wall time: 18.9 s


Create matchups

In [14]:
# Shrink datasets for faster processing
complete_dataset.drop(columns={'description', 'batterName', 'pitcherName', 'postOnFirst', 'postOnSecond', 'postOnThird', 'preOnFirst', 'preOnSecond', 'preOnThird', 'pitch_name'}, inplace=True)
complete_dataset = complete_dataset.query(f'date > {game_df["date"].astype(int).min()-10000}')
steamer_hitters_df = steamer_hitters_df.query(f'date > {game_df["date"].astype(int).min()-10000}')
steamer_pitchers_df = steamer_pitchers_df.query(f'date > {game_df["date"].astype(int).min()-10000}')

In [15]:
%%time
print(len(game_df))
empty_list = Parallel(n_jobs=-1, verbose=True)(delayed(create_matchup_files)(game_df, row, complete_dataset, steamer_hitters_df, steamer_pitchers_df, team_map) for row in range(len(game_df)))

14


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 out of  14 | elapsed:   16.7s finished


CPU times: total: 3.92 s
Wall time: 17.1 s


### B02. Simulations

In [16]:
%run "C:\Users\james\Documents\MLB\Code\B02. Simulations.ipynb"

In [17]:
num_batches = os.cpu_count()
batch_size = 63

In [18]:
if historic == False:
    # Openers
    draftables = pd.read_csv(os.path.join(baseball_path, "A01. DraftKings", "2. Draftables", f"Draftables {draftGroupId}.csv"), encoding='iso-8859-1')
    opener_list = list(draftables.query('Salary <= 5500 and `Roster Position` == "P"')['Name'])

    # Teams with Projected Lineups
    projected_lineups = pd.read_csv(os.path.join(baseball_path, "A05. Rosters", "3. Batting Orders Projected", f"Batting Orders Projected {todaysdate}.csv"))
else:
    opener_list, projected_lineups = [], None

In [19]:
# Choose which games to run simulations for
if slate_only == True:
    # Just the slate
    run_df = slate_df.copy()
else:
    # All games
    run_df = game_df.copy()

In [20]:
%%time
# Print out games to simulate
_ = [print(f"{row['away_team']}@{row['home_team']} {pd.to_datetime(row['game_datetime']).tz_convert('US/Eastern').strftime('%Y-%m-%d %H:%M:%S')}") for index, row in run_df.head(15).iterrows()]

# Loop over dates of interest
for date in list(run_df['date'].unique()):
    print(date)
    
    # Extract year
    year = date[:4]
    # Extract matchups
    matchup_list = [os.path.splitext(f)[0] for f in os.listdir(os.path.join(baseball_path, "B01. Matchups", f"Matchups {date}"))]
    
    # Loop over games
    daily_df = run_df.query(f'date == "{date}"').reset_index()
    print(daily_df.shape)
    for i in range(len(daily_df)):            
        # Extract info from run_df to look up proper file in matchups path
        away_team = daily_df['away_team'][i]
        home_team = daily_df['home_team'][i]
        game_id = daily_df['game_id'][i]
        # Beginning of filename in matchup folder
        lookup = f"{away_team}@{home_team} {game_id}"
        
        # Find the matchup file
        matchup = next((matchup for matchup in matchup_list if matchup.startswith(lookup)), None)
        print(matchup)
        
        # Create matchups
        matchup_path = os.path.join(baseball_path, "B01. Matchups", f"Matchups {date}", f"{matchup}.xlsx")
        AwayBatters, HomeBatters, AwayPitchers, HomePitchers = create_matchup(matchup_path, batter_stats_scaler, batter_stats_fg_scaler, batter_imputations_model, pitcher_stats_scaler, pitcher_stats_fg_scaler, pitcher_imputations_model, projected_lineups)

        # Read in weather
        weather_df = pd.read_csv(os.path.join(baseball_path, "A06. Weather", "4. Park and Weather Factors", f"Park and Weather Factors {date}.csv"))
        weather_df = weather_df.query(f'game_id == {game_id}').reset_index(drop=True)
        
        # Create Scoreboard object
        game_template = Scoreboard(AwayBatters, HomeBatters, AwayPitchers, HomePitchers, 9)       
        
        ### Sim games
        # With batching (fast!)
        start = time.time()
        game_list = Parallel(n_jobs=num_batches, verbose=1)(delayed(sim_game_batch)(game_template, model_pulls, model_binary, model_outs, model_safe, opener_list, weather_df, innings=9, debug=False, batch_size=batch_size) for batches in range(num_batches))
        game_list = [game for sublist in game_list for game in sublist]
        print(f"Simming {batch_size*num_batches} games took {time.time() - start} seconds.")
        
        # # Without joblib (slow!)
        # start = time.time()
        # game_list = []
        # for i in range(10):
        #     game = sim_game(game_template, model_pulls, model_binary, model_outs, model_safe, opener_list, weather_df, innings=9, debug=True)
        #     game_list.append(game)
        # print(f"Simming {num_sims} games took {time.time() - start} seconds.")
        
        # Create object path
        game_path = os.path.join(baseball_path, "B02. Simulations", "1. Game Sims", f"Matchups {date}", f"{matchup}") 
        player_path = os.path.join(baseball_path, "B02. Simulations", "2. Player Sims", f"Matchups {date}", f"{matchup}") 
        # Clear folders
        if os.path.exists(game_path):
            shutil.rmtree(game_path)
        if os.path.exists(player_path):
            shutil.rmtree(player_path)
        # Make folders
        os.makedirs(game_path, exist_ok=True)
        os.makedirs(player_path, exist_ok=True)
        
        # Save each object in the list to a separate file
        for i, game_object in enumerate(game_list):
            # Delete unnecessary attributes from batter objects
            for batter in game_object.away_batters + game_object.home_batters:
                batter = batter.keep_selected_attributes()
            # Delete unnecessary attributes from pitcher objects
            for pitcher in game_object.away_pitchers + game_object.home_pitchers:
                pitcher = pitcher.keep_selected_attributes()
            # Delete unnecessary attributes from game objects
            game_object.keep_selected_attributes()

            # Constructing DataFrames for game, batters, and pitchers
            game_score_df = pd.DataFrame({
                "away_score": [game_object.away_score],
                "home_score": [game_object.home_score]
            })

            away_batters_df = pd.DataFrame([vars(batter) for batter in game_object.away_batters])
            away_batters_df['team'] = "away"
            home_batters_df = pd.DataFrame([vars(batter) for batter in game_object.home_batters])
            home_batters_df['team'] = "home"
            batters_df = pd.concat([away_batters_df, home_batters_df], axis=0)
            
            away_pitchers_df = pd.DataFrame([vars(pitcher) for pitcher in game_object.away_pitchers])
            away_pitchers_df['team'] = "away"
            home_pitchers_df = pd.DataFrame([vars(pitcher) for pitcher in game_object.home_pitchers])
            home_pitchers_df['team'] = "home"
            pitchers_df = pd.concat([away_pitchers_df, home_pitchers_df], axis=0)
            
            # Specify the file paths where you want to save the CSV files
            game_file_path = os.path.join(game_path, "game_{}.csv").format(i)
            batters_file_path = os.path.join(player_path, "batters_{}.csv").format(i)
            pitchers_file_path = os.path.join(player_path, "pitchers_{}.csv").format(i)

            # Saving DataFrames to CSV files
            game_score_df.to_csv(game_file_path, index=False)
            batters_df.to_csv(batters_file_path, index=False)
            pitchers_df.to_csv(pitchers_file_path, index=False)

        print(f'Saved to {game_path}')

BAL@TOR 2024-08-06 19:07:00
MIL@ATL 2024-08-06 19:20:00
TBR@STL 2024-08-06 19:45:00
HOU@TEX 2024-08-06 20:05:00
MIN@CHC 2024-08-06 20:05:00
BOS@KCR 2024-08-06 20:10:00
NYM@COL 2024-08-06 20:40:00
DET@SEA 2024-08-06 21:40:00
CHW@OAK 2024-08-06 21:40:00
PHI@LAD 2024-08-06 22:10:00
20240806
(10, 33)
BAL@TOR 744901 1907


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:   37.2s remaining:  4.4min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:   38.8s finished


Simming 1008 games took 38.87738108634949 seconds.
Saved to C:\Users\james\Documents\MLB\Database\B02. Simulations\1. Game Sims\Matchups 20240806\BAL@TOR 744901 1907
MIL@ATL 747086 1920
Spots in away batting order are imputed.


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:   39.6s remaining:  4.6min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:   41.2s finished


Simming 1008 games took 41.337870597839355 seconds.
Saved to C:\Users\james\Documents\MLB\Database\B02. Simulations\1. Game Sims\Matchups 20240806\MIL@ATL 747086 1920
TBR@STL 745141 1945


...

Simming 1008 games took 40.68068504333496 seconds.
Saved to C:\Users\james\Documents\MLB\Database\B02. Simulations\1. Game Sims\Matchups 20240806\DET@SEA 745221 2140
CHW@OAK 745629 2140
Away batters use Baseball Monster projected lineups
Home batters use Baseball Monster projected lineups


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:   38.8s remaining:  4.5min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:   40.7s finished


Simming 1008 games took 40.877113342285156 seconds.
Saved to C:\Users\james\Documents\MLB\Database\B02. Simulations\1. Game Sims\Matchups 20240806\CHW@OAK 745629 2140
PHI@LAD 746120 2210
Away batters use Baseball Monster projected lineups
Home batters use Baseball Monster projected lineups
Spots in home batting order are imputed.


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:   39.2s remaining:  4.6min
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:   40.9s finished


Simming 1008 games took 41.10653758049011 seconds.
Saved to C:\Users\james\Documents\MLB\Database\B02. Simulations\1. Game Sims\Matchups 20240806\PHI@LAD 746120 2210
CPU times: total: 44.9 s
Wall time: 8min 3s


### B03. Lineups

In [21]:
%run "B03. Optimizer.ipynb"
%run "C02. Projections.ipynb"

In [22]:
# Lineup ranking method
sort_by = 'Plus3'

# Set maximum ownership by position group
max_exposure_batters = 0.5
max_exposure_pitchers = 0.7

# Lineups to create
lineups = 100

### Run One

##### 1. Players

In [23]:
draftables_with_sims = create_player_file(contestKey, guide, draftGroupId, roto_slate, max_exposure_pitchers, max_exposure_batters)
draftables_with_sims.to_csv(os.path.join(baseball_path, "B03. Lineups", "1. Players", f"Players {contestKey}.csv"), index=False, encoding='iso-8859-1')

In [24]:
# webbrowser.open(os.path.join(baseball_path, "B03. Lineups", "1. Players", f"Players {contestKey}.csv"))

##### 2. Lineups

In [25]:
constraint_sets = {
"Maximum": [(49000, 5, 5, 2, red_list+orange_list, 10), (49000, 5, 5, 1, red_list+orange_list, 10)],
"Medium": [(49000, 5, 5, 2, red_list, 10), (49000, 5, 5, 1, red_list, 10)],
"Minimum": [(49000, 5, 5, 2, red_list, 0), (49000, 5, 5, 1, red_list, 0), (49000, 5, 5, 3, red_list, 0), (49000, 4, 4, 1, red_list, 0), (49000, 4, 3, 1, red_list, 0)]
}

In [26]:
%%time
for constraint_set in constraint_sets:
    try:
        print(f"Attempting {constraint_set} Constraints")
        run_optimizers_parallel(contestKey, constraint_sets[constraint_set], lineups)
        break  # Exit the loop if the function succeeds
    except Exception as e:  # Correct the exception type to 'Exception'
        print(f"Attempt failed: {e}\n")


Attempting Maximum Constraints
Attempt failed: create_lineups() takes from 1 to 7 positional arguments but 8 were given

Attempting Medium Constraints
Attempt failed: create_lineups() takes from 1 to 7 positional arguments but 8 were given

Attempting Minimum Constraints
Attempt failed: create_lineups() takes from 1 to 7 positional arguments but 8 were given

CPU times: total: 0 ns
Wall time: 2 ms


##### 3. Lineups Ranked

In [27]:
sort_by_list = ['P50', 'P75', 'P90', 'P95', 'P99', 'P100', 'Tail', 'Sim STD', 'Plus2', 'Plus3', 'rostership', 'pitcher rostership', 'batter_rostership']
lineups_ranked = choose_lineups(contestKey, roto_slate, sort_by)
lineups_ranked.to_csv(os.path.join(baseball_path, "B03. Lineups", "3. Lineups Ranked", f"Lineups Ranked {contestKey}.csv"), index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\james\\Documents\\MLB\\Database\\B03. Lineups\\2. Lineups\\Lineups 164597148.csv'

In [ ]:
# webbrowser.open(os.path.join(baseball_path, "B03. Lineups", "3. Lineups Ranked", f"Lineups Ranked {contestKey}.csv"))

In [ ]:
# Goal: Just calculate dollars won by each, take average (this is if you generate lineups = contest size) hard to do for large contests

In [ ]:
# Filter columns starting with 'Sim '
sim_columns = [col for col in lineups_ranked.columns if col.startswith('Sim ')]

# Initialize the Wins and Top1% columns
lineups_ranked['Wins'] = 0
lineups_ranked['Top1%'] = 0
lineups_ranked['Top5%'] = 0
lineups_ranked['Top10%'] = 0
lineups_ranked['Top20%'] = 0
lineups_ranked['Top50%'] = 0

# Iterate over each 'Sim ' column
for col in sim_columns:
    # Find the maximum value in the column
    max_value = lineups_ranked[col].max()
    # Increment the 'Wins' count for rows with the maximum value in this column
    lineups_ranked.loc[lineups_ranked[col] == max_value, 'Wins'] += 1
    
    # Calculate the top 1% threshold for the current column
    top_1_percent_threshold = lineups_ranked[col].quantile(0.99)
    top_5_percent_threshold = lineups_ranked[col].quantile(0.95)
    top_10_percent_threshold = lineups_ranked[col].quantile(0.90)
    top_20_percent_threshold = lineups_ranked[col].quantile(0.80)
    top_50_percent_threshold = lineups_ranked[col].quantile(0.50)
    # Increment the 'Top1%' count for rows with values in the top 1%
    lineups_ranked.loc[lineups_ranked[col] >= top_1_percent_threshold, 'Top1%'] += 1
    lineups_ranked.loc[lineups_ranked[col] >= top_5_percent_threshold, 'Top5%'] += 1
    lineups_ranked.loc[lineups_ranked[col] >= top_10_percent_threshold, 'Top10%'] += 1
    lineups_ranked.loc[lineups_ranked[col] >= top_20_percent_threshold, 'Top20%'] += 1
    lineups_ranked.loc[lineups_ranked[col] >= top_50_percent_threshold, 'Top50%'] += 1

# Convert the Top1% count to a percentage
lineups_ranked['Top1%'] = (lineups_ranked['Top1%'] / len(sim_columns)) * 100
lineups_ranked['Top5%'] = (lineups_ranked['Top5%'] / len(sim_columns)) * 100
lineups_ranked['Top10%'] = (lineups_ranked['Top10%'] / len(sim_columns)) * 100
lineups_ranked['Top20%'] = (lineups_ranked['Top20%'] / len(sim_columns)) * 100
lineups_ranked['Top50%'] = (lineups_ranked['Top50%'] / len(sim_columns)) * 100


In [ ]:
lineups_ranked.query('pareto == 1')

In [ ]:
lineups_ranked.sort_values('Wins', ascending=False).head(10)

In [ ]:
lineups_ranked.reset_index(inplace=True)

In [ ]:
import pandas as pd
import plotly.express as px

# Assuming lineups_ranked is your DataFrame
# lineups_ranked = pd.read_csv('your_data.csv')  # Example of loading data

# Scatter plot of Plus3 vs batter_rostership with hover data from multiple columns
fig = px.scatter(
    lineups_ranked,
    x='Plus3',
    y='batter rostership',
    color='pareto',
    hover_data={
        'index': True,
        'P': True,
        'P.1': True,
        '1B': True,
        '2B': True,
        '3B': True,
        'SS': True,
        'OF': True,
        'OF.1': True,
        'OF.2': True,
        'Wins': True
    },
    title='Scatter Plot of Plus3 vs Batter Rostership',
    labels={'Plus3': 'Plus3', 'batter rostership': 'Batter Rostership'}
)

# Update the size of the figure and invert the y-axis
fig.update_layout(
    width=1000,  # Set the width of the plot
    height=800,  # Set the height of the plot
    yaxis=dict(autorange='reversed')  # Invert the y-axis
)

fig.update_traces(marker=dict(size=10))

fig.show()


##### 4. Uploads

In [ ]:
upload = create_upload_file(contestKey, sort_by)
upload.to_csv(os.path.join(baseball_path, "B03. Lineups", "4. Uploads", f"Upload {contestKey}.csv"), index=False)

##### 5. Entries

In [ ]:
entry = create_entry_file(draftGroupId, contestKey)
entry.to_csv(os.path.join(baseball_path, "B03. Lineups", "5. Entries", f"Entries {draftGroupId}.csv"), index=False, encoding='iso-8859-1')

##### Upload

In [ ]:
upload_entries(draftGroupId)

##### Email

In [ ]:
email_upload_file(draftGroupId, contestKey)

### Run All

In [ ]:
# # Create list of contestKeys with available guides
# contestKey_list = os.listdir(os.path.join(baseball_path, "A09. Contest Guides"))
# contestKey_list = [int(key.replace(".csv", "").split(" ")[2]) for key in contestKey_list]


# # Create list of contestKeys with available results
# result_contestKey_list = os.listdir(os.path.join(baseball_path, "A01. DraftKings", "6. Player Results"))
# result_contestKey_list = [int(key.replace(".csv", "").split(" ")[2]) for key in result_contestKey_list]

# # Find the overlap 
# available_contestKey_list = list(set(result_contestKey_list) & set(contestKey_list))

In [ ]:
# %%time
# fail_list = Parallel(n_jobs=-1, verbose=1)(delayed(create_contest_lineups2)(contestKey, sort_by, min_salary, min_projection[0], major_stack[0], minor_stack[0], excluded_teams=[], lineups, historic=True) for contestKey in available_contestKey_list)

In [ ]:
# %%time
# fail_list2 = Parallel(n_jobs=-1, verbose=1)(delayed(create_contest_lineups2)(contestKey, sort_by='Plus3', min_salary=49000, min_projection=4, major_stack=5, minor_stack=2, max_exposure_batters=0.5, max_exposure_pitchers=0.75, excluded_teams=[], lineups=lineups, historic=True) for contestKey in [contestKey for contestKey in fail_list if contestKey is not None])

In [ ]:
# %%time
# fail_list3 = Parallel(n_jobs=-1, verbose=1)(delayed(create_contest_lineups2)(contestKey, sort_by='Plus3', min_salary=49000, min_projection=4, major_stack=4, minor_stack=2, max_exposure_batters=0.5, max_exposure_pitchers=0.75, excluded_teams=[], lineups=lineups, historic=True) for contestKey in [contestKey for contestKey in fail_list2 if contestKey is not None])